In [74]:
from pathlib import Path
import pandas as pd
import json, ast, re

RAW  = Path("raw_data_csv") / "yelp_academic_dataset_business.csv"
OUT  = Path("clean_data_business");  OUT.mkdir(exist_ok=True)

def write(df, name):
    path = OUT / f"{name}.csv"
    df.to_csv(path, index=False)
    print(f"{name:24s} → {len(df):,} rows  ({path})")

def parse_dict_like(val):
    if isinstance(val, str) and re.match(r"\s*\{.*\}\s*$", val):
        try: return ast.literal_eval(val)
        except Exception: pass
    return {}

pd.set_option('display.max_columns', None)

In [60]:
df = (
    pd.read_csv(RAW, dtype=str)
      .dropna(how="all", axis=1)
      .drop_duplicates(subset=["business_id"])
)

print(df.shape)
df.head()

(150346, 60)


,address,attributes,attributes.AcceptsInsurance,attributes.AgesAllowed,attributes.Alcohol,attributes.Ambience,attributes.BYOB,attributes.BYOBCorkage,attributes.BestNights,attributes.BikeParking,attributes.BusinessAcceptsBitcoin,attributes.BusinessAcceptsCreditCards,attributes.BusinessParking,attributes.ByAppointmentOnly,attributes.Caters,attributes.CoatCheck,attributes.Corkage,attributes.DietaryRestrictions,attributes.DogsAllowed,attributes.DriveThru,attributes.GoodForDancing,attributes.GoodForKids,attributes.GoodForMeal,attributes.HairSpecializesIn,attributes.HappyHour,attributes.HasTV,attributes.Music,attributes.NoiseLevel,attributes.Open24Hours,attributes.OutdoorSeating,attributes.RestaurantsAttire,attributes.RestaurantsCounterService,attributes.RestaurantsDelivery,attributes.RestaurantsGoodForGroups,attributes.RestaurantsPriceRange2,attributes.RestaurantsReservations,attributes.RestaurantsTableService,attributes.RestaurantsTakeOut,attributes.Smoking,attributes.WheelchairAccessible,attributes.WiFi,business_id,categories,city,hours,hours.Friday,hours.Monday,hours.Saturday,hours.Sunday,hours.Thursday,hours.Tuesday,hours.Wednesday,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,"1616 Chapala St, Ste 2","{""ByAppointmentOnly"":""True""}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pns2l4eNsfO8kk83dixA6A,"Doctors, Traditional Chinese Medicine, Naturop...",Santa Barbara,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,34.4266787,-119.7111968,"Abby Rappoport, LAC, CMQ",93101,7,5.0,CA
1,87 Grasso Plaza Shopping Center,"{""BusinessAcceptsCreditCards"":""True""}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mpf3x-BjTdTEA3yCZrAYPw,"Shipping Centers, Local Services, Notaries, Ma...",Affton,"{""Monday"":""0:0-0:0"",""Tuesday"":""8:0-18:30"",""Wed...",8:0-18:30,0:0-0:0,8:0-14:0,NaN,8:0-18:30,8:0-18:30,8:0-18:30,1,38.551126,-90.335695,The UPS Store,63123,15,3.0,MO
2,5255 E Broadway Blvd,"{""BikeParking"":""True"",""BusinessAcceptsCreditCa...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,True,"{'garage': False, 'street': False, 'validated'...",False,False,False,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,False,NaN,NaN,NaN,False,NaN,NaN,False,NaN,2,False,NaN,False,NaN,True,u'no',tUFrWirKiKi_TAnsVWINQQ,"Department Stores, Shopping, Fashion, Home & G...",Tucson,"{""Monday"":""8:0-22:0"",""Tuesday"":""8:0-22:0"",""Wed...",8:0-23:0,8:0-22:0,8:0-23:0,8:0-22:0,8:0-22:0,8:0-22:0,8:0-22:0,0,32.223236,-110.880452,Target,85711,22,3.5,AZ
3,935 Race St,"{""RestaurantsDelivery"":""False"",""OutdoorSeating...",NaN,NaN,u'none',NaN,NaN,NaN,NaN,True,NaN,False,"{'garage': False, 'street': True, 'validated':...",False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,False,NaN,1,NaN,NaN,True,NaN,NaN,u'free',MTSW4McQd7CbVtyjqoe9mw,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",Philadelphia,"{""Monday"":""7:0-20:0"",""Tuesday"":""7:0-20:0"",""Wed...",7:0-21:0,7:0-20:0,7:0-21:0,7:0-21:0,7:0-20:0,7:0-20:0,7:0-20:0,1,39.9555052,-75.1555641,St Honore Pastries,19107,80,4.0,PA
4,101 Walnut St,"{""BusinessAcceptsCreditCards"":""True"",""Wheelcha...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,True,"{'garage': None, 'street': None, 'validated': ...",NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,True,NaN,mWMc6_wTdE0EUBKIGXDVfA,"Brewpubs, Breweries, Food",Green Lane,"{""Wednesday"":""14:0-22:0"",""Thursday"":""16:0-22:0...",12:0-22:0,NaN,12:0-22:0,12:0-18:0,16:0-22:0,NaN,14:0-22:0,1,40.3381827,-75.4716585,Perkiomen Valley Brewery,18054,13,4.5,PA


In [61]:
num_cols = ["latitude", "longitude", "stars"]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df["review_count"] = pd.to_numeric(df["review_count"], downcast="integer", errors="coerce")
df["is_open"]      = df["is_open"].astype("Int8")
df[num_cols + ["review_count", "is_open"]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   latitude      150346 non-null  float64
 1   longitude     150346 non-null  float64
 2   stars         150346 non-null  float64
 3   review_count  150346 non-null  int16  
 4   is_open       150346 non-null  Int8   
dtypes: Int8(1), float64(3), int16(1)
memory usage: 4.0 MB


In [62]:
cat_df = (
    df[["business_id", "categories"]]
      .dropna()
      .assign(categories=lambda x: x["categories"].str.split(r",\s*"))
      .explode("categories")
      .rename(columns={"categories": "category"})
)
write(cat_df, "business_category")

df = df.drop(columns="categories")

business_category        → 668,592 rows  (clean_data_business\business_category.csv)


In [ ]:
# most likely don't need these tables
hours_dict = (
    df.pop("hours")
      .apply(lambda s: json.loads(s) if pd.notna(s) else {})
)

hours_wide = (
    hours_dict.apply(pd.Series)
              .rename(columns=lambda c: f"hours.{c}")
              .assign(business_id=df["business_id"])
)
write(hours_wide, "business_hours_wide")

hours_tall = (
    hours_wide
      .melt(id_vars="business_id", var_name="weekday", value_name="hours")
      .dropna(subset=["hours"])
      .assign(weekday=lambda x: x["weekday"].str.replace("hours.", "", regex=False))
)
write(hours_tall, "business_hour")

KeyError: 'hours'

In [ ]:
# pull BusinessParking out <--- this is also most likely unnecessary
if "attributes.BusinessParking" in df.columns:
    parking_series = df.pop("attributes.BusinessParking").apply(parse_dict_like)
    parking_df = (
        pd.json_normalize(parking_series)
          .add_prefix("parking.")
          .assign(business_id=df["business_id"])
    )
    write(parking_df, "business_parking")

# build fresh list of remaining attributes
attr_cols = [c for c in df.columns if c.startswith("attributes.")]

if attr_cols:
    # melt as before
    attr_df = (
        df.melt(id_vars=["business_id"],
                value_vars=attr_cols,
                var_name="attr_key",
                value_name="attr_value")
          .dropna(subset=["attr_value"])
    )

    # strip prefixes
    attr_df["attr_key"] = (
        attr_df["attr_key"]
          .str.replace("attributes.", "", regex=False)
    )

    # split group / subkey
    splits = attr_df["attr_key"].str.split(".", n=1, expand=True)
    attr_df["attr_group"] = splits[0]
    if splits.shape[1] > 1:
        attr_df["attr_sub"] = splits[1]
    else:
        attr_df["attr_sub"] = ""

    # normalize boolean/null strings and numerics
    _map = {
        "true": True,  "false": False,
        "yes": True,   "no": False,
        "1": True,     "0": False,
        "-1": False,
        "none": pd.NA, "null": pd.NA, "": pd.NA,
    }
    val = (
        attr_df["attr_value"]
          .astype(str)
          .str.strip()
          .str.lower()
    )
    attr_df["attr_value_clean"] = (
        val.map(_map)
           .where(val.isin(_map), attr_df["attr_value"])
    )

    num_mask = (
        attr_df["attr_value_clean"]
          .astype(str)
          .str.fullmatch(r"\d+(\.\d+)?")
    )
    attr_df.loc[num_mask, "attr_value_clean"] = pd.to_numeric(
        attr_df.loc[num_mask, "attr_value_clean"]
    )

    # write
    write(attr_df, "business_attributes")
    df = df.drop(columns=attr_cols)
else:
    print("No attributes.* columns to melt.")

business_parking         → 150,346 rows  (clean_data_business\business_parking.csv)
business_attributes      → 1,105,179 rows  (clean_data_business\business_attributes.csv)


In [65]:
atts = pd.read_csv("clean_data_business/business_attributes.csv", dtype=str)

# find rows whose value is a dict- or list-like string
mask_dict = atts["attr_value_clean"].str.match(r"^\s*\{.*\}\s*$", na=False)
mask_list = atts["attr_value_clean"].str.match(r"^\s*\[.*\]\s*$", na=False)
mask_nested = mask_dict | mask_list

print("Nested keys:", atts.loc[mask_nested, "attr_key"].unique())

# separate and parse nested rows
nested = atts[mask_nested].copy()
nested["parsed"] = nested["attr_value_clean"].apply(ast.literal_eval)

# flatten each parsed object into a tall table of new key/value rows
rows = []
for _, r in nested.iterrows():
    biz = r["business_id"]
    key = r["attr_key"]
    val = r["parsed"]
    if isinstance(val, dict):
        for subk, subv in val.items():
            rows.append({"business_id": biz,
                         "attr_group": key,
                         "attr_sub": subk,
                         "attr_value_clean": subv})
    elif isinstance(val, list):
        for item in val:
            rows.append({"business_id": biz,
                         "attr_group": key,
                         "attr_sub": None,
                         "attr_value_clean": item})

expanded = pd.DataFrame(rows)

# build final attribute df
base = atts[~mask_nested].drop(columns=["parsed"], errors="ignore")
final_attr = pd.concat([
    base,
    expanded[["business_id","attr_group","attr_sub","attr_value_clean"]]
], ignore_index=True)
def normalize(v):
    if pd.isna(v):
        return pd.NA

    s = str(v).strip()

    # u'…' or surrounding single‐quotes
    m = re.match(r"^u?'(.+)'$", s)
    if m:
        s = m.group(1)

    if s.isdigit():
        return int(s)

    low = s.lower()
    if low in ("true", "yes"):
        return True
    if low in ("false", "no"):
        return False

    return low.capitalize()

# apply normalization
final_attr["attr_value_clean"] = final_attr["attr_value_clean"].apply(normalize)
final_attr.drop('attr_value', axis=1, inplace=True)
# re-write back out
final_attr.to_csv("clean_data_business/business_attributes.csv", index=False)

Nested keys: ['Ambience' 'BestNights' 'DietaryRestrictions' 'GoodForMeal'
 'HairSpecializesIn' 'Music']


In [66]:
write(df, "business_clean")
# drop the attributes col
df=df.drop(columns='attributes', axis=1)

# column re-ordering
biz = df.pop('business_id')
df.insert(0, 'business_id', biz)
days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
df = df[[c for c in df.columns if not c.startswith('hours.')]
      + [f'hours.{d}' for d in days if f'hours.{d}' in df.columns]]

front = ['business_id','name','address','city','state','postal_code']
rest  = [c for c in df.columns if c not in front]
new_order = front + rest
idx       = df.columns.get_indexer(new_order)
df = df.iloc[:, idx]
df.head()

business_clean           → 150,346 rows  (clean_data_business\business_clean.csv)


,business_id,name,address,city,state,postal_code,is_open,latitude,longitude,review_count,stars,hours.Monday,hours.Tuesday,hours.Wednesday,hours.Thursday,hours.Friday,hours.Saturday,hours.Sunday
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,0,34.426679,-119.711197,7,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,1,38.551126,-90.335695,15,3.0,0:0-0:0,8:0-18:30,8:0-18:30,8:0-18:30,8:0-18:30,8:0-14:0,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,0,32.223236,-110.880452,22,3.5,8:0-22:0,8:0-22:0,8:0-22:0,8:0-22:0,8:0-23:0,8:0-23:0,8:0-22:0
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,1,39.955505,-75.155564,80,4.0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-21:0,7:0-21:0,7:0-21:0
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,1,40.338183,-75.471659,13,4.5,NaN,NaN,14:0-22:0,16:0-22:0,12:0-22:0,12:0-22:0,12:0-18:0


In [67]:
df.to_csv("clean_data_business/business_clean.csv", index=False)

In [72]:
df = pd.read_csv("clean_data_business/business_clean.csv")
df.head()

,business_id,name,address,city,state,postal_code,is_open,latitude,longitude,review_count,stars,hours.Monday,hours.Tuesday,hours.Wednesday,hours.Thursday,hours.Friday,hours.Saturday,hours.Sunday
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,0,34.426679,-119.711197,7,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,1,38.551126,-90.335695,15,3.0,0:0-0:0,8:0-18:30,8:0-18:30,8:0-18:30,8:0-18:30,8:0-14:0,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,0,32.223236,-110.880452,22,3.5,8:0-22:0,8:0-22:0,8:0-22:0,8:0-22:0,8:0-23:0,8:0-23:0,8:0-22:0
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,1,39.955505,-75.155564,80,4.0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-21:0,7:0-21:0,7:0-21:0
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,1,40.338183,-75.471659,13,4.5,NaN,NaN,14:0-22:0,16:0-22:0,12:0-22:0,12:0-22:0,12:0-18:0


In [69]:
atts = pd.read_csv(r'C:\Users\xtm_j\OneDrive\Desktop\talk_of_the_town\clean_data_business\business_attributes.csv')
atts

C:\Users\xtm_j\AppData\Local\Temp\ipykernel_4588\1454793103.py:1: DtypeWarning: Columns (1,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  atts = pd.read_csv(r'C:\Users\xtm_j\OneDrive\Desktop\talk_of_the_town\clean_data_business\business_attributes.csv')


,business_id,attr_key,attr_group,attr_sub,attr_value_clean
0,Kq51_lGAgAigqmENITTr-A,AcceptsInsurance,AcceptsInsurance,NaN,False
1,E_kLUlU60fzoZYAOxbH_3w,AcceptsInsurance,AcceptsInsurance,NaN,True
2,M_THnNDHs29aylcK-JqS1A,AcceptsInsurance,AcceptsInsurance,NaN,True
3,t5tBd4p7BA7hNDitBUXxrw,AcceptsInsurance,AcceptsInsurance,NaN,True
4,2oxrOO3c9_mQmqM9kwEm9Q,AcceptsInsurance,AcceptsInsurance,NaN,True
...,...,...,...,...,...
1680999,w_4xUt-1AyY2ZwKtnjW0Xg,NaN,Music,no_music,False
1681000,w_4xUt-1AyY2ZwKtnjW0Xg,NaN,Music,jukebox,False
1681001,w_4xUt-1AyY2ZwKtnjW0Xg,NaN,Music,live,False
1681002,w_4xUt-1AyY2ZwKtnjW0Xg,NaN,Music,video,False


In [ ]:
atts['attr_key'].unique()

array(['AcceptsInsurance', 'AgesAllowed', 'Alcohol', 'BYOB',
       'BYOBCorkage', 'BikeParking', 'BusinessAcceptsBitcoin',
       'BusinessAcceptsCreditCards', 'ByAppointmentOnly', 'Caters',
       'CoatCheck', 'Corkage', 'DogsAllowed', 'DriveThru',
       'GoodForDancing', 'GoodForKids', 'HappyHour', 'HasTV',
       'NoiseLevel', 'Open24Hours', 'OutdoorSeating', 'RestaurantsAttire',
       'RestaurantsCounterService', 'RestaurantsDelivery',
       'RestaurantsGoodForGroups', 'RestaurantsPriceRange2',
       'RestaurantsReservations', 'RestaurantsTableService',
       'RestaurantsTakeOut', 'Smoking', 'WheelchairAccessible', 'WiFi',
       nan], dtype=object)

In [ ]:
atts['attr_group'].unique()

array(['AcceptsInsurance', 'AgesAllowed', 'Alcohol', 'BYOB',
       'BYOBCorkage', 'BikeParking', 'BusinessAcceptsBitcoin',
       'BusinessAcceptsCreditCards', 'ByAppointmentOnly', 'Caters',
       'CoatCheck', 'Corkage', 'DogsAllowed', 'DriveThru',
       'GoodForDancing', 'GoodForKids', 'HappyHour', 'HasTV',
       'NoiseLevel', 'Open24Hours', 'OutdoorSeating', 'RestaurantsAttire',
       'RestaurantsCounterService', 'RestaurantsDelivery',
       'RestaurantsGoodForGroups', 'RestaurantsPriceRange2',
       'RestaurantsReservations', 'RestaurantsTableService',
       'RestaurantsTakeOut', 'Smoking', 'WheelchairAccessible', 'WiFi',
       'Ambience', 'BestNights', 'DietaryRestrictions', 'GoodForMeal',
       'HairSpecializesIn', 'Music'], dtype=object)

In [ ]:
atts['attr_sub'].unique()

array([nan, 'romantic', 'intimate', 'touristy', 'hipster', 'divey',
       'classy', 'trendy', 'upscale', 'casual', 'monday', 'tuesday',
       'friday', 'wednesday', 'thursday', 'sunday', 'saturday',
       'dairy-free', 'gluten-free', 'vegan', 'kosher', 'halal',
       'soy-free', 'vegetarian', 'dessert', 'latenight', 'lunch',
       'dinner', 'brunch', 'breakfast', 'straightperms', 'coloring',
       'extensions', 'africanamerican', 'curly', 'kids', 'perms', 'asian',
       'dj', 'background_music', 'no_music', 'jukebox', 'live', 'video',
       'karaoke'], dtype=object)

In [ ]:
atts['attr_value_clean'].unique()

array(['False', 'True', '21plus', 'Allages', '18plus', nan, 'Full_bar',
       'Beer_and_wine', 'Yes_free', 'Yes_corkage', False, True, 'Average',
       'Quiet', 'Loud', 'Very_loud', 'Casual', 'Formal', 'Dressy', '2',
       '3', '4', 'Outdoor', 'Free', 'Paid'], dtype=object)